In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import pickle
import numpy as np
import sys
import gzip
from function import *
import click
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from tqdm import tqdm

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

plt.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 14,
    #'xtick.labelsize': 12,
    #'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.dpi': 300,  # backup if you don't specify dpi in savefig
})

In [ ]:
def photon_counter(wf, template, endpoint, tick, debug = False):
    wf = np.array(wf)
    template = np.array(template)
    template = template - np.median(template)

    if (np.max(wf[:150])) < 0:
        return -1, -1

    try:
        template_half_max = np.max(template) / 2
        wf_half_max = np.max(wf[:150]) / 2
        template_idx = np.where(template >= template_half_max)[0][0]
    except:
        return -1, -1
    wf_idx = np.where(wf[:150] >= wf_half_max)[0][0]
    delta = template_idx - wf_idx

    wf = np.roll(wf, delta)
    wf = wf[:len(template)]
    #wf = wf - np.mean(wf[:80])
    wf = wf[100:]
    template = template[100:]

    plt.plot(wf/np.max(wf))
    plt.plot(template/np.max(template))
    plt.show()
    
    if endpoint == 109:
        lim = 500
    else:
        if len(template) > 400:
            lim = 400
        else:
            lim = 200
        
#try:
    params, errors, r_squared = conv_fit_v5(wf, template, 150, lim, factor = 2.5, debug = debug)

    if r_squared < 0.5:
        return -1, -1
    else:
    
        A_s, A_t, tau_s, tau_t, offset = params
        eA_s, eA_t,  etau_s, etau_t,  eoffset = errors
    
        area   = A_s*tau_s*(1-np.exp(-16*tick/(tau_s))) + (A_t*tau_t*(1-np.exp(-16*tick/(tau_t))))
        e_area = np.sqrt((A_s*etau_s)**2 + (tau_s*eA_s)**2 + (A_t*etau_t)**2 + (tau_t*eA_t)**2)

        # for plot 
        time = 16*np.arange(0, len(wf))
        new_time = np.linspace(time[0], time[-1], int(2.5 * len(time)))
        template_interp = interp1d(time, template, kind='linear')
        template_upsampled = template_interp(new_time)
        dt = (new_time[1] - new_time[0])

        lAr  = scintillation(new_time, A_s, A_t, tau_s, tau_t)
        conv = fft_convolution(new_time, *params, template_upsampled, dt)

        plt.figure(figsize=(6,4), constrained_layout=True)
        plt.plot(time, wf, 'blue', label = 'Signal')
        plt.plot(new_time, conv, 'k', label = 'Fit')

        info_text = (
            f"$A_s$ = {A_s:.2f} ± {eA_s:.2f}\n"
            f"$\\tau_s$ (ns) = {tau_s:.1f} ± {etau_s:.1f}\n"
            f"$A_t$ = {A_t:.3f} ± {eA_t:.3f}\n"
            f"$\\tau_t$ (ns) = {int(tau_t)} ± {int(etau_t)}\n"
            f"$N_{{PE}}$ = {int(area)} ± {int(e_area)}\n"
            f"R$^2$ = {r_squared:.3f}"
        )

        # Add the text box to the plot
        ax = plt.gca()
        box = AnchoredText(
            info_text, loc='upper right',
            frameon=True, bbox_transform=ax.transAxes, borderpad=0.5
        )
        ax.add_artist(box)

        plt.xlim(0, 16*len(time))
        plt.xlabel('Time(ns)')
        plt.ylabel('ADCs')
        plt.legend(loc = 'lower right')
        plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.6)
        plt.show()
        return area, e_area
    #except:
    #    return -1, -1
        

In [ ]:
# Template
plt.figure(figsize=(6,4), constrained_layout=True)
with open(f'Templates/v_27xxx_fix_avg/Template - 104_10.pkl', 'rb') as file:
    t = pickle.load(file)
plt.plot(t)
plt.xlim(0, len(t))
plt.xlabel('Ticks')
plt.ylabel('ADCs')
plt.title(f'SPE Template: Endpoint 104 channel 10')

plt.show()
template = t - np.median(t)

In [ ]:
# Signal

plt.figure(figsize=(6,4), constrained_layout=True)
with open(f'cutted_inverted_baseline_wf_adcs.pkl', 'rb') as file:
    w = pickle.load(file)
plt.plot(w)
plt.xlim(0, len(w))
plt.xlabel('Ticks')
plt.ylabel('ADCs')
plt.title(f'Waveform: Endpoint 104 channel 10')


In [ ]:
photon_counter(w, t, 104, 256, debug = False)